In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Ustawiamy ziarno losowe (seed), żeby wyniki były powtarzalne
tf.random.set_seed(42)
np.random.seed(42)

# Konwersja danych wejściowych z tensora PyTorch/TF na numpy arrays 
# i rzutowanie na odpowiednie typy (int32 dla kategorii, float32 dla liczb zmiennoprzecinkowych).
# Dzięki temu model w Kerasie dostaje dane w formacie, którego oczekuje.
cat_train_np = cat_train.numpy().astype('int32')
cat_test_np = cat_test.numpy().astype('int32')
con_train_np = con_train.numpy().astype('float32')
con_test_np = con_test.numpy().astype('float32')
y_train_np = y_train.numpy().astype('float32')
y_test_np = y_test.numpy().astype('float32')

# Przygotowanie słownika wejść dla modelu:
# - każdy atrybut kategoryczny dostaje osobny "wektor kolumnowy"
# - dane ciągłe grupujemy razem w jednej macierzy pod kluczem 'cont'
train_inputs = {f'cat_{i}': cat_train_np[:, i:i+1] for i in range(cat_train_np.shape[1])}
test_inputs = {f'cat_{i}': cat_test_np[:, i:i+1] for i in range(cat_test_np.shape[1])}
train_inputs['cont'] = con_train_np
test_inputs['cont'] = con_test_np

# Informacyjnie wypisujemy wersję TensorFlow (pomocne przy debugowaniu i odtwarzaniu wyników)

inputs_cat = []     # lista wejść dla zmiennych kategorycznych
embeddings = []     # lista embeddingów odpowiadających tym wejściom

# Tworzymy osobne wejście i warstwę embedding dla każdej cechy kategorycznej
for idx, (cardinality, emb_dim) in enumerate(emb_szs):
    # Wejście (pojedyncza liczba całkowita reprezentująca kategorię)
    input_layer = keras.Input(shape=(1,), name=f'cat_{idx}')
    
    # Warstwa embedding: zamiana indeksu kategorii na gęsty wektor o wymiarze emb_dim
    embed = keras.layers.Embedding(cardinality, emb_dim, name=f'emb_{idx}')(input_layer)
    
    # Reshape: spłaszczenie (bo Embedding zwraca kształt (batch, 1, emb_dim))
    embed = keras.layers.Reshape((emb_dim,), name=f'emb_{idx}_reshape')(embed)
    
    # Zbieramy do list
    embeddings.append(embed)
    inputs_cat.append(input_layer)

embeddings, inputs_cat


NameError: name 'cat_train' is not defined

In [3]:
# łączymy je w jeden wektor cech
cat_features = keras.layers.Concatenate(name='cats_concat')(embeddings)


# Dropout dla embeddingów — redukuje przeuczenie
cat_features = keras.layers.Dropout(0.4, name='cats_dropout')(cat_features)

# Wejście dla zmiennych ciągłych
cont_input = keras.Input(shape=(con_train_np.shape[1],), name='cont')

# BatchNormalization — normalizacja danych ciągłych (przyspiesza uczenie)
cont_features = keras.layers.BatchNormalization(name='cont_bn')(cont_input)

# Połączenie cech kategorycznych i ciągłych w jeden wektor
x = keras.layers.Concatenate(name='features_concat')([cat_features, cont_features])

# Klasyczne gęste warstwy ukryte (MLP)
for i, units in enumerate([200, 100]):
    # Gęsta warstwa w pełni połączona z ReLU
    x = keras.layers.Dense(units, activation='relu', name=f'dense_{i}')(x)
    # Normalizacja batchowa stabilizuje rozkład aktywacji
    x = keras.layers.BatchNormalization(name=f'bn_{i}')(x)
    # Dropout dla redukcji przeuczenia
    x = keras.layers.Dropout(0.4, name=f'dropout_{i}')(x)

# Warstwa wyjściowa — 1 neuron (predykcja ceny przejazdu)
output = keras.layers.Dense(1, name='fare')(x)

# Składamy cały model — wejścia to embeddingi + cechy ciągłe
tf_model = keras.Model(inputs=inputs_cat + [cont_input], outputs=output, name='taxi_fare_model_tf')

# Kompilacja modelu:
# - optimizer Adam (0.001)
# - funkcja straty: MSE (błąd średniokwadratowy)
# - metryka: RMSE (bardziej interpretowalna w jednostkach ceny)
tf_model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='mse',
    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')]
)

# Podsumowanie modelu (lista warstw, parametry, kształty)
tf_model.summary()

NameError: name 'embeddings' is not defined